In [21]:
import pdb
import json
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('pdf',fonttype = 42)
plt.rc('ps',fonttype = 42)
import seaborn as sns

sns.set_style('whitegrid')
sns.set_context('talk', font_scale=0.9)
# import seaborn as sns
# sns.set_context("talk") #, font_scale=1.1)

# rdir = '../results/lpc/results_22-05-13r1/'
# rdir = '../results/lpc/results_22-05-14/'
rdirs = [
#     '../results/lpc/results_22-05-16/'
#     '../results/lpc/results_22-05-14r1/',
#     '../results_23-01-27/',
    '../results/results_23-01-30/',
        ]
figdir='../overleaf/figs/'
def save(fig, name):
#     fig.tight_layout()
    for t in ['.pdf','.jpg']:
        plt.savefig(figdir+name+t, 
                    dpi=300,
                    bbox_inches='tight'
                   )

In [2]:
# nice names
nice_names = {
    'roc_auc_test':'AUROC',
    'auprc_test':'AUPRC',
    'MC_loss_test':'MC loss',
    'PMC_loss_test':'PMC loss',
    'DC_loss_test':'DC loss',
    
}
nice_levels = {
    'algorithm':{
        'lr':'LR',
        'lr_mc':'LR+MC',
        'lr_mc_cv':'LR+MC-CV',
        'lr_pmc':'LR+PMC',
        'lr_pmc_log':'LR+PMC-log',
        'lr_pmc_cv':'LR+PMC-CV',
        'rf':'RF',
        'rf_mc':'RF+MC',
        'rf_mc_cv':'RF+MC-CV',
        'rf_pmc':'RF+PMC',
        'rf_pmc_log':'RF+PMC-log',
        'rf_pmc_cv':'RF+PMC_CV'
    }
}

In [25]:
with open('../experiment/data/mimic4_admissions.csv.label_encodings.json') as f:
    labels = json.load(f)
print(labels)

def recode_labels(df):
    for k,v in labels.items():
        if k in df.columns:
            try:
                df[k] = df[k].apply(lambda x: v['classes_'][x])
            except Exception as e:
                print(e.what())
                pdb.set_trace()
            
    return df
    
# d = get_calibration_df('../results_23-01-24/mimic4_admissions_rf_pmc_ohc_4322__2023-01-25_17-16-40.csv')
# recode_labels(d)

{'insurance': {'classes_': ['Medicaid', 'Medicare', 'Other']}, 'language': {'classes_': ['?', 'ENGLISH']}, 'marital_status': {'classes_': ['DIVORCED', 'MARRIED', 'SINGLE', 'WIDOWED', nan]}, 'ethnicity': {'classes_': ['AMERICAN INDIAN/ALASKA NATIVE', 'ASIAN', 'BLACK/AFRICAN AMERICAN', 'HISPANIC/LATINO', 'OTHER', 'UNABLE TO OBTAIN', 'UNKNOWN', 'WHITE']}, 'gender': {'classes_': ['F', 'M']}}


In [26]:
from sklearn.metrics import roc_curve, auc, brier_score_loss
from sklearn.calibration import calibration_curve
n_bins = 10
bins = np.linspace(float(1.0/n_bins), 1.0, n_bins)
bins[0] = 0.0
def get_calibration_df(f):

    try:
        df = pd.read_csv(f, on_bad_lines='skip').drop(columns='Unnamed: 0')
        df = df.dropna()
    except Exception as e:
        print('!!!Failed to load',f)
        print(e)
    group = [g for g in ['ethnicity','gender','insurance'] if g in df.columns]
    
    g = f.replace('.csv','.json')
    with open(g,'r') as file:
        try:
            annot = json.load(file)
        except Exception as e:
            print('!!!Failed to load',g)
            print(e)
    frames = []
    for grp, dfg in df.groupby(group):
        try:
            y_true = dfg['y_true'].astype(int)
        except:
            pdb.set_trace()
        y_proba = dfg['y_pred_proba'].astype(float)
        fp,mpv = calibration_curve(y_true, y_proba, n_bins=n_bins)
        group_dict = {k:v for k,v in zip(group,grp)}
        group_dict.update({
            'Fraction Positive':fp,
            'mean_prediction_value':mpv
        })
        dfnew = pd.DataFrame(group_dict)
        dfnew['brier_score_loss'] = brier_score_loss(y_true, y_proba)
        frames.append(dfnew)
    df = pd.concat(frames) #.melt(id_vars=group)
    for k,v in annot.items():
        if not isinstance(v, list):
            df[k] = v
    return recode_labels(df)



In [24]:
frames = []
# cols = [
#     'algorithm',
#     'random_state',
#     'time_time',
#        ]
globs = []
for rdir in rdirs:
    globs.extend(glob(rdir+'/*.csv'))
for f in globs:
    d = get_calibration_df(f)
     
    frames.append(d)    
    
df_results = pd.concat(frames)
print(len(frames),'records')
# print(frames[0].keys())

df_results['ml_name'] = df_results['algorithm'].apply(lambda x: x.split('_pmc')[0].split('_mc')[0])
df_results['postprocessing'] = df_results['algorithm'].apply(lambda x: 'MC-CV' if '_mc_cv' in x else x)
df_results['postprocessing'] = df_results['postprocessing'].apply(lambda x: 'MC' if '_mc' in x else x)
df_results['postprocessing'] = df_results['postprocessing'].apply(lambda x: 'PMC-CV' if '_pmc_cv' in x else x)
df_results['postprocessing'] = df_results['postprocessing'].apply(lambda x: 'PMC' if '_pmc' in x else x)
df_results['postprocessing'] = df_results['postprocessing'].apply(lambda x: '-' if not x in ['MC','PMC','MC-CV','PMC-CV'] else x)
df_results.loc[df_results['ml_name']=='lr_cv','ml_name'] = 'lr'
df_results['Algorithm'] = df_results['algorithm'].apply(lambda x: nice_levels['algorithm'][x])
df_results = df_results.loc[df_results.n_bins==10]
# df_results = df_results.loc[df_results.gamma==0.05]
df_results.groupby('algorithm')['random_state'].count()

> /tmp/ipykernel_3358526/2533052993.py(13)recode_labels()
     11         pdb.set_trace()
     12 
---> 13     return df
     14 
     15 # d = get_calibration_df('../results_23-01-24/mimic4_admissions_rf_pmc_ohc_4322__2023-01-25_17-16-40.csv')

ipdb> print(k)
ethnicity
ipdb> print(v)
{'classes_': ['AMERICAN INDIAN/ALASKA NATIVE', 'ASIAN', 'BLACK/AFRICAN AMERICAN', 'HISPANIC/LATINO', 'OTHER', 'UNABLE TO OBTAIN', 'UNKNOWN', 'WHITE']}
ipdb> df[k]
0          0
1          0
0          0
1          0
2          0
      ...   
5          7
6          7
7          7
8          7
0    2100843
Name: ethnicity, Length: 356, dtype: int64
ipdb> df[k].value_counts()
2          54
7          52
3          50
4          50
1          47
6          45
5          30
0          27
2100843     1
Name: ethnicity, dtype: int64
ipdb> df[k].apply(lambda x: v['classes_'][x])
*** NameError: name 'v' is not defined
ipdb> v['classes_'][7]
'WHITE'
ipdb> v['classes_'][x]
*** NameError: name 'x' is not defined
ipdb

In [ ]:
df_results

In [ ]:
df_results[['alpha','gamma']].value_counts()

In [ ]:
import seaborn as sns
n_bins = 10
bins = np.linspace(0, 1.0, n_bins+1)
# bin_centers = [np.mean(bins[i:i+1]) for i,_ in enumerate(bins[:-1])]
bin_centers = [0.1, .3, .5, .7, .9]
df_plt = df_results.copy()
df_plt = df_results.loc[~df_results.ethnicity.isin([
   'UNKNOWN',
   'OTHER',
   'UNABLE TO OBTAIN',
    'AMERICAN INDIAN/ALASKA NATIVE'
]), :]
df_plt = df_plt.loc[df_plt.algorithm.isin(['rf'])]
# df_plt = df_plt.loc[df_plt.algorithm.str.contains('rf')]
# df_plt = df_plt.loc[df_plt.algorithm=='rf']

df_plt['Prediction Bin'] = pd.cut(df_plt['mean_prediction_value'], bins = bins, include_lowest=True).apply(lambda x: x.mid)
# df_plt['mean_prediction_value'] 
g = sns.relplot(
    kind='line',
    markers=True,
    data=df_plt,
    x='Prediction Bin',
    y='Fraction Positive',
    hue='Algorithm',
#     style='gender',
#     col='ethnicity',
#     style='gender',
#     col='insurance',
#     row='insurance',
    err_style='bars',
#     row='gender',
#     scatter=False,
#     lowess=True,
#     x_estimator=np.mean
#     height=3
    
)
# for k,ax in g.axes_dict.items():
plt.xlabel('Predicted Value')
plt.gca().plot([0, 1],[0,1], color='gray', linestyle='--', lw=2)

In [ ]:
import seaborn as sns
n_bins = 10
bins = np.linspace(0, 1.0, n_bins+1)
# bin_centers = [np.mean(bins[i:i+1]) for i,_ in enumerate(bins[:-1])]
bin_centers = [0.1, .3, .5, .7, .9]
df_plt = df_results.copy()
df_plt = df_results.loc[~df_results.ethnicity.isin([
   'UNKNOWN',
   'OTHER',
   'UNABLE TO OBTAIN',
    'AMERICAN INDIAN/ALASKA NATIVE'
]), :]
# df_plt = df_plt.loc[df_plt.algorithm.str.contains('rf')]
df_plt = df_plt.loc[df_plt.algorithm.isin(['rf'])]

df_plt['Prediction Bin'] = pd.cut(df_plt['mean_prediction_value'], bins = bins, include_lowest=True).apply(lambda x: x.mid)
# df_plt['mean_prediction_value'] 
g = sns.relplot(
    kind='line',
    data=df_plt,
    markers=True,
    x='Prediction Bin',
    y='Fraction Positive',
    hue='Algorithm',
    style='gender',
    col='ethnicity',
#     row='gender',
#     col='insurance',
    row='insurance',
    err_style='bars',
#     row='gender',
#     scatter=False,
#     lowess=True,
#     x_estimator=np.mean
#     height=3
    
)
g.set_titles("{col_name}, {row_name}")
for k,ax in g.axes_dict.items():
    ax.plot([0, 1],[0,1], color='gray', linestyle='--', lw=2)

In [ ]:
n_bins = 10
bins = np.linspace(0, 1.0, n_bins+1)
# bin_centers = [np.mean(bins[i:i+1]) for i,_ in enumerate(bins[:-1])]
bin_centers = [0.1, .3, .5, .7, .9]
df_plt = df_results.copy()
df_plt = df_results.loc[~df_results.ethnicity.isin([
   'UNKNOWN',
   'OTHER',
   'UNABLE TO OBTAIN',
    'AMERICAN INDIAN/ALASKA NATIVE'
]), :]
# df_plt = df_plt.loc[df_plt.algorithm.str.contains('rf')]
df_plt = df_plt.loc[df_plt.algorithm.isin(['rf','rf_pmc'])]

df_plt['Prediction Bin'] = pd.cut(df_plt['mean_prediction_value'], bins = bins, include_lowest=True).apply(lambda x: x.mid)
# df_plt['mean_prediction_value'] 
g = sns.relplot(
    kind='line',
    markers=True,
    data=df_plt,
    x='Prediction Bin',
    y='Fraction Positive',
    hue='Algorithm',
    hue_order=['RF','RF+PMC'],
    size='Algorithm',
    sizes=(2,3),
    style='gender',
    col='ethnicity',
#     style='gender',
#     col='insurance',
    row='insurance',
    err_style='bars',
#     row='gender',
#     scatter=False,
#     lowess=True,
#     x_estimator=np.mean
#     height=3
    
)
g.set_titles("{col_name}, {row_name}")
for k,ax in g.axes_dict.items():
#     ax.set_title(ax.get_title(), fontsize=12)
    ax.plot([0, 1],[0,1], color='k', linestyle='--', lw=1)

In [ ]:
n_bins = 10
bins = np.linspace(0, 1.0, n_bins+1)
# bin_centers = [np.mean(bins[i:i+1]) for i,_ in enumerate(bins[:-1])]
bin_centers = [0.1, .3, .5, .7, .9]
df_plt = df_results.copy()
df_plt = df_results.loc[~df_results.ethnicity.isin([
   'UNKNOWN',
   'OTHER',
   'UNABLE TO OBTAIN',
    'AMERICAN INDIAN/ALASKA NATIVE'
]), :]
# df_plt = df_plt.loc[df_plt.algorithm.str.contains('rf')]
df_plt = df_plt.loc[df_plt.algorithm.isin(['lr','lr_pmc'])]

df_plt['Prediction Bin'] = pd.cut(df_plt['mean_prediction_value'], bins = bins, include_lowest=True).apply(lambda x: x.mid)
# df_plt['mean_prediction_value'] 
g = sns.relplot(
    kind='line',
    markers=True,
    data=df_plt,
    x='Prediction Bin',
    y='Fraction Positive',
    hue='Algorithm',
#     hue_order=['RF+PMC','RF+MC'],
    size='Algorithm',
    sizes=(2,3),
    style='gender',
    col='ethnicity',
    row='gender',
#     col='insurance',
#     row='insurance',
    err_style='bars',
#     row='gender',
#     scatter=False,
#     lowess=True,
#     x_estimator=np.mean
#     height=3
    
)
g.set_titles("{col_name}, {row_name}")
for k,ax in g.axes_dict.items():
#     ax.set_title(ax.get_title(), fontsize=12)
    ax.plot([0, 1],[0,1], color='k', linestyle='--', lw=1)

In [ ]:
bins

In [ ]:
import seaborn as sns

df_plt = df_results.loc[df_results.ethnicity.isin([
   'WHITE',
   'BLACK/AFRICAN AMERICAN',
   'HISPANIC/LATINO',
   'ASIAN'
]), :]
df_plt['Prediction Bin'] = pd.cut(df_plt['mean_prediction_value'], bins = bins, include_lowest=True).apply(lambda x: x.mid)
df_plt = df_plt.loc[df_plt.algorithm.isin(['rf','rf_pmc'])]
# df_plt['mean_prediction_value'] 
g = sns.lmplot(
#     kind='scatter',
    data=df_plt,
#     x='mean_prediction_value',
    x='Prediction Bin',
    y='Fraction Positive',
    hue='Algorithm',
#     style='gender',
    col='ethnicity',
    row='insurance',
#     row='algorithm',
#     row='gender',
#     scatter=False,
    lowess=True,
#     x_estimator=np.mean
    
)
for k,ax in g.axes_dict.items():
    ax.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2)

In [ ]:
import seaborn as sns

df_plt = df_results.loc[df_results.ethnicity.isin([
   'WHITE',
   'BLACK/AFRICAN AMERICAN',
   'HISPANIC/LATINO',
   'ASIAN'
]), :]
df_plt['Prediction Bin'] = pd.cut(df_plt['mean_prediction_value'], bins = bins, include_lowest=True).apply(lambda x: x.mid)
df_plt = df_plt.loc[df_plt.algorithm.isin(['rf','rf_mc'])]
# df_plt['mean_prediction_value'] 
g = sns.lmplot(
#     kind='scatter',
    data=df_plt,
#     x='mean_prediction_value',
    x='Prediction Bin',
    y='Fraction Positive',
    hue='Algorithm',
#     style='gender',
    col='ethnicity',
    row='insurance',
#     row='algorithm',
#     row='gender',
#     scatter=False,
#     lowess=True,
#     x_estimator=np.mean
    
)
for k,ax in g.axes_dict.items():
    ax.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2)